# Activations

> Module containing helper functions and classes around activations

In [ ]:
#|default_exp activations

In [ ]:
 #| export
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn
import torch.nn.functional as F
from operator import attrgetter
from functools import partial
import fastcore.all as fc
import math
import torcheval.metrics as tem
import matplotlib.pyplot as plt
import random
import numpy as np

In [ ]:
 #| export
def set_seed(seed, deterministic=False):
    torch.use_deterministic_algorithms(deterministic)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
 #| export
class Hook():
    def __init__(self, nr, layer, func):
        wrapped_func = partial(func, self) # pass the Hook object into the function
        self.hook = layer.register_forward_hook(wrapped_func)
        self.layer_name = f'{nr}_{layer.__class__.__name__}'
        
    def remove(self):
        self.hook.remove()

In [ ]:
 #| export
class ActivationStatsS(Subscriber):
    
    def __init__(self, modules):
        self.modules = modules
    
    def before_fit(self, learn):
        self.hooks = [Hook(i, module, partial(self.record_stats, learn)) for i, module in enumerate(self.modules)]
        
    def record_stats(self, learn, hook, layer, inp, outp):
        if learn.model.training:
            if not hasattr(hook, 'stats'): hook.stats = ([], [], [], [])
            acts = outp.detach().cpu()
            hook.stats[0].append(acts.mean())              # get the means over all activations
            hook.stats[1].append(acts.std())               # get the stds over all activations
            hook.stats[2].append(acts.histc(20,-10,10))    # get the histogram counts with 20 bins (-10,10)
            
            # computation of the not_firing_rate_per_activation
            N = acts.shape[0]                 
            flat = acts.view(N, -1)                        # flatten the activations: matrix of [samples, activations]
            nf_rate_p_act = (flat == 0.0).sum(dim=0) / N   # compute not firing rate per activations (so across the samples)
            hook.stats[3].append(nf_rate_p_act)   

    def after_fit(self, learn):
        for h in self.hooks: h.remove()

In [ ]:
#|export
@fc.patch()
def plot(self:ActivationStatsS, figsize=(15,4), average_firing_rate=False):
    plots = 3 if average_firing_rate else 2
    fig,axs = plt.subplots(1,plots, figsize=figsize)
    legend = []
    for h in self.hooks:
        axs[0].plot(h.stats[0])
        axs[0].set_title('mean')
        axs[1].plot(h.stats[1])
        axs[1].set_title('std')
        if average_firing_rate:
            axs[2].plot(1-torch.stack(h.stats[3]).T.mean(dim=0))
            axs[2].set_title('average firing rate')
            axs[2].set_ylim(0,1)
        legend.append(h.layer_name)
    plt.legend(legend);

@fc.patch()
def plot_hist(self:ActivationStatsS, figsize=None, log=True):
    if figsize is None: figsize = (15, len(self.hooks))
    fig,axs = plt.subplots(math.ceil(len(self.hooks)/2), 2, figsize=figsize)
    axs = axs.flat
    for i, hook in enumerate(self.hooks):
        d = torch.stack(hook.stats[2]).T
        if log: d = d.log1p()
        axs[i].imshow(d, cmap='Blues', origin='lower', aspect='auto')
        axs[i].set_title(hook.layer_name)
        axs[i].set_yticks(np.arange(0, 20, 2), np.arange(-10, 10, 2))

@fc.patch()
def plot_dead(self:ActivationStatsS, binary=False, figsize=None):
    if figsize is None: figsize = (15, len(self.hooks))
    fig,axs = plt.subplots(math.ceil(len(self.hooks)/2), 2, figsize=figsize)
    axs = axs.flat
    for i, hook in enumerate(self.hooks):
        d = torch.stack(hook.stats[3]).T
        if binary: d = d == 1.0
        axs[i].imshow(d, cmap='Greys', origin='lower', aspect='auto')
        axs[i].set_title(hook.layer_name)

In [ ]:
 #| export
def init_weights(m):
    if isinstance(m, nn.Conv2d): torch.nn.init.kaiming_normal_(m.weight)

In [ ]:
 #| export
class NormalizationS(Subscriber):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def before_batch(self, learn):
        learn.batch = [(learn.batch[0] - self.mean) / self.std, learn.batch[1]]

In [ ]:
 #| export
def conv_block(in_c, out_c, kernel_size=3, act=True, norm=True):
    padding = kernel_size // 2
    stride = 2
    layers = [torch.nn.Conv2d(in_c, out_c, kernel_size, stride, padding, bias=not norm)]
    if norm: layers.append(torch.nn.BatchNorm2d(out_c))
    if act: layers.append(torch.nn.ReLU())
    return nn.Sequential(*layers) if len(layers)>1 else layers[0]

In [ ]:
 #| export
def cnn_layers(act=True):
    return nn.Sequential(                  
        conv_block(1 , 8, kernel_size=5),
        conv_block(8 ,16),
        conv_block(16,32),
        conv_block(32,64),
        conv_block(64,10, norm=False, act=False),
        nn.Flatten())